In [1]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input

In [2]:


def multiply(A,B):
    return tf.math.floormod(tf.linalg.matvec(A,B),n_colors)

def cvec(my_list):
    return K.reshape(my_list,(-1,1))

def rvec(my_list):
    return K.reshape(my_list,(1,-1))

def mat(my_list):
    n = int(np.floor(np.sqrt(len(my_list))))
    return K.reshape(my_list,(n,n))

In [3]:
def generate_data(n_vertices,n_colors,sample_size):
#     off = np.zeros((n_vertices,1),dtype=int)
#     on = np.ones((n_vertices,1),dtype=int)
    train_size = int(np.floor(0.8*sample_size))
    X_train = np.zeros((train_size,n_vertices**2))
    Y_train = tf.convert_to_tensor(K.ones((train_size,n_vertices)))
    X_test = np.zeros((sample_size-train_size,n_vertices**2))
    Y_test = tf.convert_to_tensor(K.ones((sample_size-train_size,n_vertices)))
    for idx in range(0,sample_size):
        adjacency_matrix = np.zeros((n_vertices,n_vertices),dtype=int)
        for i in range(0,n_vertices):
            for j in range(0,n_vertices):
                if i==j:
                    adjacency_matrix[i,j]=1
                elif i<j:
                    adjacency_matrix[i,j]=np.random.randint(0,n_colors)
                elif i>j:
                    adjacency_matrix[i,j]=adjacency_matrix[j,i]
        am=tf.convert_to_tensor(adjacency_matrix)
        if idx < train_size:
            X_train[idx] = rvec(am)[0]
        
        else:
            X_test[idx-train_size] = rvec(am)[0]
            
        X_train_final = tf.convert_to_tensor(X_train)
        X_test_final = tf.convert_to_tensor(X_test)
    
    
    
    return (X_train_final,Y_train,X_test_final,Y_test)    

                

In [4]:
def loss_function(flat_matrix,n_vertices):
    def error(true,predicted):
        batch_size = 1
        matrices = tf.reshape(flat_matrix,(-1,n_vertices,n_vertices))
        predicted_vectors = tf.reshape(predicted,(-1,n_vertices))
        true_vectors = tf.reshape(true,(-1,n_vertices))
        total = 0
        for i in range(batch_size):
            matrix = matrices[i]
            pred_vec = predicted_vectors[i]
            true_vec = true_vectors[i]
            output = multiply(matrix,pred_vec)
            difference = true_vec-output
            discrepancy = tf.map_fn(abs,difference)
            total += tf.math.reduce_sum(discrepancy)**2

        return total/batch_size
        
    
    return error;

def score(inputs):
    count = 0
    total = len(inputs)
    for i in range(total):
        vec = cvec(model.predict(inputs).round()[i])
        a=tf.cast(mat(X_train[i]),dtype=tf.int64)
        b=tf.cast(vec,dtype=tf.int64)[:,0]
        if list(multiply(a,b).numpy())==[1 for i in range(n_vertices)]:
            count+=1
    return count/total

In [16]:
n_vertices = 6
n_colors = 2
bs = 32
epochs = 100
X_train,Y_train,X_test,Y_test = generate_data(n_vertices,n_colors,500)

def fit_model():
    input_tensor = Input(shape=(n_vertices**2,))
    layer1 = Dense(100, activation='relu')(input_tensor)
    out = Dense(n_vertices, activation='relu')(layer1)
    model = Model(input_tensor, out)
    model.compile(loss=loss_function(input_tensor,n_vertices), optimizer='adam',experimental_run_tf_function=False)
    
    model.fit(
        X_train,
        Y_train,
        epochs=epochs,
        shuffle=True,
        verbose=2,
        batch_size=bs
    )
    
    return model;

In [17]:
model=fit_model()

Train on 400 samples
Epoch 1/100
400/400 - 0s - loss: 11.2603
Epoch 2/100
400/400 - 0s - loss: 8.1866
Epoch 3/100
400/400 - 0s - loss: 8.1754
Epoch 4/100
400/400 - 0s - loss: 8.4716
Epoch 5/100
400/400 - 0s - loss: 8.2363
Epoch 6/100
400/400 - 0s - loss: 8.5845
Epoch 7/100
400/400 - 0s - loss: 9.3466
Epoch 8/100
400/400 - 0s - loss: 8.6885
Epoch 9/100
400/400 - 0s - loss: 5.9116
Epoch 10/100
400/400 - 0s - loss: 6.9752
Epoch 11/100
400/400 - 0s - loss: 7.5652
Epoch 12/100
400/400 - 0s - loss: 6.9683
Epoch 13/100
400/400 - 0s - loss: 6.6195
Epoch 14/100
400/400 - 0s - loss: 4.9964
Epoch 15/100
400/400 - 0s - loss: 6.5343
Epoch 16/100
400/400 - 0s - loss: 5.4271
Epoch 17/100
400/400 - 0s - loss: 6.6026
Epoch 18/100
400/400 - 0s - loss: 6.5441
Epoch 19/100
400/400 - 0s - loss: 6.9276
Epoch 20/100
400/400 - 0s - loss: 4.9035
Epoch 21/100
400/400 - 0s - loss: 5.0416
Epoch 22/100
400/400 - 0s - loss: 5.4363
Epoch 23/100
400/400 - 0s - loss: 4.1445
Epoch 24/100
400/400 - 0s - loss: 6.5903
Epo

In [18]:
print(f"Percentage Correct (Training Data): {score(X_train)}")
print(f"Percentage Correct (Testing Data): {score(X_test)}")

Percentage Correct (Training Data): 0.07
Percentage Correct (Testing Data): 0.03


In [ ]:
?model.fit

Source: https://stackoverflow.com/questions/46464549/keras-custom-loss-function-accessing-current-input-pattern
